# Bilan Hydrique

Sources :

- Coefficients culturaux :
  - [ARDEPI](https://www.ardepi.fr/nos-services/vous-etes-irrigant/estimer-ses-besoins-en-eau/maraichage/)
  - [Chambre d’agriculture Nouvelle-Aquitaine](https://gironde.chambre-agriculture.fr/fileadmin/user_upload/Nouvelle-Aquitaine/100_Inst-Gironde/Documents/pdf_grandes-cultures_accompagnement-technique_mieux-irriguer/Messages_irrigation_2019/message_1/Tableau_Coefficients_Culturaux_Kc_.02.pdf)


In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
# Constantes
# Coefficients culturaux (KC) par culture et par stade
KC = {
    'Pomme de terre': {'Vegetation': 0.9, 'Maximale': 1.05}
}

# Réserve Utile (RU) par cm de terre fine (mm/cm de terre fine) en fonction de la texture du sol
RU_PAR_CM_DE_TF = {
    'Terres argileuses': 1.85,
    'Argiles sableuses': 1.7, 'Argiles sablo-limoneuses': 1.8, 'Argiles limono-sableuses': 1.8, 'Argiles limoneuses': 1.9,
    'Terres argilo-sableuses': 1.7, 'Terres argilo-limono-sableuses': 1.8, 'Terres argilo-limoneuses': 2.,
    'Terres sablo-argileuses': 1.4, 'Terres sablo-limono-argileuses': 1.5, 'Terres limono-sablo-argileuses': 1.65, 'Terres limono-argileuses': 2.00,
    'Terres sableuses': 0.7, 'Terres sablo-limoneuses': 1., 'Terres limono-sableuses': 1.55, 'Terres limoneuses': 1.8
}

# Profondeurs d'enracinement typiques
PROFONDEUR_ENRACINEMENT_TYPIQUE = {
    'Radis': 15.,
    'Salade': 15.,
    'Choux': 20.,
    'Epinard': 20.,
    'Oignon': 20.,
    'Aubergine': 30.,
    'Carotte': 30.,
    'Courge': 30.,
    'Courgette': 30.,
    'Poivron': 30.,
    'Pomme de terre': 30.,
    'Tomate': 30.
}

In [3]:
# Données météorologiques
# Évapotranspiration Potentielle (ETP) sur la période (mm)
ETP = 28.

# Réserve Facilement Utilisable (RFU) initiale (mm)
RFU_INITIALE = 40.

# Pluviométrie (mm)
PLUVIOMETRIE = 5.

In [4]:
# RFU finale cible (mm)
RFU_FINALE_CIBLE = 40.

# Coefficient de conversion de la RU en RFU (entre 1/2 et 2/3)
RU_VERS_RFU = 2. / 3

# Fraction de la réserve utile du sol remplie d'eau (entre 0 pour une période sèche et 1 pour une période pluvieuse)
FRACTION_REMPLIE = 1.

# Fraction du sol occupé par des cailloux et graviers (entre 0 pour absence de cailloux et 1 pour totalité de cailloux)
FRACTION_CAILLOUX = 0.3

In [5]:
# Choix de la culture
culture = 'Pomme de terre'

# Choix du stade
stade = 'Vegetation'

# Choix de la texture
texture = 'Terres limoneuses'

In [6]:
# RU par cm de terre fine pour cette texture
ru_par_cm_de_tf_texture = RU_PAR_CM_DE_TF[texture]

# Calcul de la profondeur de terre fine
profondeur = PROFONDEUR_ENRACINEMENT_TYPIQUE[culture]
profondeur_tf = profondeur * (1. - FRACTION_CAILLOUX)

# Calcul de la RU (mm)
ru = ru_par_cm_de_tf_texture * profondeur_tf

# Calcul de la RFU maximale (mm)
rfu_max = ru * RU_VERS_RFU

# Calcul de la RFU disponible (mm)
rfu = rfu_max * FRACTION_REMPLIE

print(f"RFU pour une profondeur d'enracinement de {profondeur:.0f} cm : {rfu:.0f} mm")

RFU pour une profondeur d'enracinement de 30 cm : 25 mm


In [7]:
# KC de la culture pour ce stade
kc_culture = KC[culture][stade]

# Calcul de l'évalotranspiration maximale (mm)
etm_culture = kc_culture * ETP

print(f'Évapotranspiration maximale pour la {culture} au stade {stade}: {etm_culture:.0f} (mm)')

Évapotranspiration maximale pour la Pomme de terre au stade Vegetation: 25 (mm)


In [8]:
# Calcul de la RFU finale
besoin_irrigation = RFU_FINALE_CIBLE + etm_culture - (rfu + PLUVIOMETRIE)

print(f'Besoin en irrigation pour la {culture} au stade {stade}: {besoin_irrigation:.0f} (mm)')

Besoin en irrigation pour la Pomme de terre au stade Vegetation: 35 (mm)


In [9]:
# Définition de la station de référence
REF_STATION_NAME = 'La Petite Claye'
REF_STATION_LATLON = [48.541356, -1.615400]
# Altitude (m)
REF_STATION_ALTITUDE = 50.

# Identification de l'API Météo-France
APPLICATION_ID = 'ZlFGb1VCNzdlQ3c5QmhSMU1IbE8xQTluOE0wYTpUS3l1YkcweGJmSTJrQlJVaGNiSkNHTXczdHNh'

# NN_NOMBRE = 20
# NN_RAYON_KM = None
NN_NOMBRE = None
NN_RAYON_KM = 35.

In [10]:
import meteofrance

# Initialisation d'un client pour accéder à l'API Météo-France
client = meteofrance.Client(APPLICATION_ID)

# Issue a series of API requests an example. For use this test, you must first subscribe to the arome api with your application
client.session.headers.update({'Accept': '*/*'})

In [11]:
df_liste_stations = meteofrance.demande_liste_stations(client)

/home/atantet/.conda/envs/agri_notebooks/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'portail-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/atantet/.conda/envs/agri_notebooks/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [12]:
import geo

df_liste_stations_nn = geo.selection_plus_proches_voisins(
    df_liste_stations, REF_STATION_LATLON, meteofrance.LATLON_LABELS,
    nombre=NN_NOMBRE, rayon_km=NN_RAYON_KM)

/home/atantet/Documents/Travail/1_agri/agri-notebooks/geo.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_liste_stations_nn.loc[:, 'distance'] = dist_km


In [13]:
print(f'{REF_STATION_NAME} ({REF_STATION_LATLON[0]:.6f}, {REF_STATION_LATLON[1]:.6f}) est à :')
for k, (nn_id, nn_series) in enumerate(df_liste_stations_nn.transpose().items()):
    station_name = df_liste_stations_nn.loc[nn_id, 'Nom_usuel']
    print(f'- {df_liste_stations_nn.loc[nn_id, 'distance']:02d} km du {k}NN {station_name} '
          f'({nn_series[meteofrance.LATLON_LABELS[0]]:.6f}, '
          f'{nn_series[meteofrance.LATLON_LABELS[1]]:.6f})')

La Petite Claye (48.541356, -1.615400) est à :
- 06 km du 0NN BROUALAN (48.485667, -1.640833)
- 09 km du 1NN PONTORSON (48.585667, -1.505167)
- 17 km du 2NN PLERGUER (48.524833, -1.843667)
- 24 km du 3NN FEINS  SA (48.326833, -1.596833)
- 27 km du 4NN PLESDER (48.406833, -1.924833)
- 29 km du 5NN MEZIERES-SUR-C. (48.308833, -1.439000)
- 31 km du 6NN ST OVIN (48.682500, -1.248667)
- 33 km du 7NN GRANVILLE (48.834500, -1.613667)
- 34 km du 8NN DINARD (48.584833, -2.076333)


In [14]:
# Liste des variables météorologiques utilisées dans les calculs d'ETP et de bilan hydrique
LISTE_VARIABLES_METEO = ['ff', 't', 'u', 'ray_glo01', 'rr1']

df_meteo = meteofrance.compiler_donnee_des_departements(
    client, df_liste_stations_nn)[LISTE_VARIABLES_METEO]

/home/atantet/.conda/envs/agri_notebooks/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/atantet/.conda/envs/agri_notebooks/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [15]:
df_meteo_ref = geo.interpolation_inverse_distance_carre(
    df_meteo, df_liste_stations_nn['distance'])

In [16]:
from etp import calcul_etp

etp = calcul_etp(df_meteo_ref, *REF_STATION_LATLON, REF_STATION_ALTITUDE)
df_meteo_ref['etp'] = etp

In [17]:
# Fonction pour aggrégé chaque variable météorologique
VARIABLE_AGGREGATORS = {
    'ff': lambda x: x.mean(0),
    't': lambda x: x.mean(0),
    'u': lambda x: x.mean(0),
    'ray_glo01': lambda x: x.sum(0),
    'rr1': lambda x: x.sum(0),
    'etp': lambda x: x.sum(0),
}
# Calcul des valeurs journalières des variables météo
s_meteo_ref_jour = pd.Series(dtype=float)
for variable, series in df_meteo_ref.items():
    s_meteo_ref_jour.loc[variable] = VARIABLE_AGGREGATORS[variable](df_meteo_ref[variable])
s_meteo_ref_jour

ff           2.993812e+00
t            2.733639e+02
u            8.609897e+01
ray_glo01    2.108000e+06
rr1          1.451993e+00
etp          3.563706e-01
dtype: float64